In [1]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    # demke - fill in as we see what state gets generated in this page.
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    #''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()



In [2]:
appdir=os.getenv('HOME')
appdir=appdir + "/sync"
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/sync " + appdir)

bash = BashSession(cwd=appdir)

In [3]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/livelock.c", 
    lang="", 
    number=True,
    title="<b>livelock.c - Full source code for example of livelock due to open file limit.</b>",
    h="100%", 
    w="100%"
) + '</font>'))


<font size="1.2rem"><b>livelock.c - Full source code for example of livelock due to open file limit.</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <stdio.h>
1: #include <stdlib.h>
2: #include <unistd.h>
3: #include <string.h>
4: #include <errno.h>
5: #include <sys/stat.h>
6: #include <sys/types.h>
7: #include <fcntl.h>
8: #include <pthread.h>
9: 
10: char template1[] = "chk/in.XXXXXX";
11: char template2[] = "chk/out.XXXXXX"; 
12: 
13: struct args_s {
14:     int id;
15:     char *infile;
16:     char *outfile;
17: };
18: 
19: /* Thread function -- livelocks if open file limit is too low. */
20: void *threadfunc(void *arg)
21: {
22:     struct args_s *my_args = (struct args_s *)arg;
23: 
24:     int fd_in;
25:     int fd_out;
26: 
27:     /* Open input file, retrying if there are too many files already open */
28:     for ( ; ; ) {
29:         fd_in = open(my_args->infile, O_RDONLY);
30:         if (fd_in > 0) {
31:             break;
32:         }
33:         if (errno != EMFILE) {
34:             return NULL;
35:         }
36:     }
37: 
38:     fprintf(stderr,"Thread %d has input file %s open now.\n", 
39:             my_args->id, my_args->infile);
40: 
41:     /* Open output file, retrying if there are too many files already open */
42:     for ( ; ; ) {
43:         fd_out = open(my_args->outfile, O_RDWR);
44:         if (fd_out > 0) {
45:             break;
46:         }
47:         if (errno != EMFILE)  {
48:             close(fd_in);
49:             return NULL;
50:         }
51:     }
52:     fprintf(stderr,"Thread %d has output file %s open now.\n",
53:             my_args->id, my_args->outfile);
54: 
55:     /* Sleep for a while to simulate time reading input and producing output */
56:     sleep(1);
57: 
58:     close(fd_in);
59:     close(fd_out);
60: 
61:     return NULL;
62: }
63: /* End of thread function */
64: 
65: int main(int argc, char **argv)
66: {
67:     int num_created = 0;
68:     int i;
69:     pthread_t ids[16];
70:     struct args_s *targs[16];
71:     char *tmpname = (char *)malloc(20);
72: 
73:     /* Create all the input and output files before creating any threads */
74:     for (i=0; i < 16; i++) {
75:         targs[i] = (struct args_s *)malloc(sizeof(struct args_s));
76:         targs[i]->id = i;
77:         memset(tmpname, 0, 20);
78:         strncpy(tmpname, template1, 20);
79:         int fd = mkstemp(tmpname);
80:         if (fd < 0) {
81:             perror("main thread mkstemp infile");
82:             break;
83:         }
84:         close(fd);
85:         targs[i]->infile = (char *)malloc(strlen(tmpname)+1);
86:         strncpy(targs[i]->infile, tmpname, strlen(tmpname)+1);
87: 
88:         memset(tmpname, 0, 20);
89:         strncpy(tmpname, template2, 20);
90:         fd = mkstemp(tmpname);
91:         if (fd < 0) {
92:             perror("main thread mkstmp outfile");
93:             break;
94:         }
95:         close(fd);
96:         targs[i]->outfile = (char *)malloc(strlen(tmpname)+1);
97:         strncpy(targs[i]->outfile, tmpname, strlen(tmpname)+1);
98:     }
99: 
100:     for (i=0; i < 16; i++) {
101:         /* Create the thread, passing input and output names */
102:         pthread_create(&ids[i], NULL, threadfunc, (void *)targs[i]);
103:     }
104: 
105:     for (i = 0; i < 16; i++) {
106:         pthread_join(ids[i], 0);
107:     }
108: 
109:         
110: }
111: 

```


</div>
</font>